## Configure local LLM coz OpenAI rate limits suck!

In [1]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.prompts import PromptTemplate
from huggingface_hub import login
from typing import Any, Dict
from dotenv import load_dotenv
import os
load_dotenv()

token = os.environ["HF_API_KEY"]
login(token)
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=token,
    torch_dtype="auto",
    device_map="auto",
)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=20)
hf_llm = HuggingFacePipeline(pipeline=pipe)

2025-01-26 23:57:16.343815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737935836.361088   18155 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737935836.366401   18155 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [2]:
import PyPDF2
import llama_parse
from datetime import datetime
from llama_parse import LlamaParse
from all_imports import *
import nest_asyncio
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext, load_index_from_storage

nest_asyncio.apply()

parser = LlamaParse(
    api_key=os.environ["LLAMA_INDEX_API_KEY"],  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="text",  # "markdown" and "text" are available,
    chunk_size="paragraph",
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
)

# sync
documents = parser.load_data("beige_book_pdfs/BeigeBook_20240117.pdf")

Error while parsing the file 'beige_book_pdfs/BeigeBook_20240117.pdf': [Errno 2] No such file or directory: '/home/ubuntu/temp/temporal_rag/beige_book_pdfs/BeigeBook_20240117.pdf'


In [28]:
storage_context = StorageContext.from_defaults(persist_dir="storage")
documents = storage_context.docstore.docs
documents = list(documents.values())

In [29]:
### Code to test the local LLM inference
template = """
Extract a list of significant events from the following document:

<start of document>
{document}
<end of document>
"""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf_llm

question = "What is electroencephalography?"

response = chain.invoke({"document": documents[9]})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
